In [ ]:
import pandas as pd
from functools import partial
import numpy as np
from rebuild_cohort import build_cohort
import demographics_utils as demo_util

In [ ]:
dset_analytical_echo = pd.read_csv(COHORT_DATA_FILE)
dset_analytical_echo["enet"] = dset_analytical_echo.echonet_prediction.notna()
dset_analytical_echo["ult_c"] = dset_analytical_echo.ultromics_prediction.notna() & (
    dset_analytical_echo.ultromics_classification != "Uncertain"
)

# Echo vars

In [ ]:
dset_analytical_echo["EF_cat"] = pd.cut(
    dset_analytical_echo.EF, [-np.inf, 40, 50, np.inf], right=False
)

# # PWT
# Cutoffs for female
dset_analytical_echo.loc[dset_analytical_echo.Sex == "F", "PWT_cat"] = pd.cut(
    dset_analytical_echo.loc[
        dset_analytical_echo.Sex == "F", "PWT"
    ],
    bins=[6, 10, 13, 16, np.inf],
    labels=["Normal", "Mild", "Moderate", "Severe"],
    ordered=True,
    right=False,
)
# Cutoffs for male
dset_analytical_echo.loc[dset_analytical_echo.Sex == "M", "PWT_cat"] = pd.cut(
    dset_analytical_echo.loc[
        dset_analytical_echo.Sex == "M", "PWT"
    ],
    bins=[6, 11, 14, 17, np.inf],
    labels=["Normal", "Mild", "Moderate", "Severe"],
    ordered=True,
    right=False,
)

# # LVH by IVS
# Cutoffs for female
dset_analytical_echo.loc[dset_analytical_echo.Sex == "F", "IVS_cat"] = pd.cut(
    dset_analytical_echo.loc[dset_analytical_echo.Sex == "F", "IVS_d_2D_calc"],
    bins=[0.6, 1.0, 1.3, 1.6, np.inf],
    labels=["Normal", "Mild", "Moderate", "Severe"],
    ordered=True,
    right=False,
)
# Cutoffs for male
dset_analytical_echo.loc[dset_analytical_echo.Sex == "M", "IVS_cat"] = pd.cut(
    dset_analytical_echo.loc[dset_analytical_echo.Sex == "M", "IVS_d_2D_calc"],
    bins=[0.6, 1.1, 1.4, 1.7, np.inf],
    labels=["Normal", "Mild", "Moderate", "Severe"],
    ordered=True,
    right=False,
)

# Relative wall category 
dset_analytical_echo["RWT_cat"] = dset_analytical_echo.RWT > 0.57

# Main
## Table 2 Demo

In [ ]:
def demo_table(df: pd.DataFrame, missing_pred: str = None) -> pd.DataFrame:
    num_dset = partial(
        demo_util.numerical,
        df=df.loc[df.enet & df.ult_c],
        missing_pred=missing_pred,
    )
    cat_dset = partial(
        demo_util.categorical_var,
        df=df.loc[df.enet & df.ult_c],
        missing_pred=missing_pred,
    )

    demo_table_list = [
        num_dset("Age"),
        cat_dset("Sex"),
        cat_dset("Race"),
        cat_dset("EF_cat"),
        num_dset("PWT"),
        cat_dset("PWT_cat").sort_index(level=1, key=demo_util.sorter),
        num_dset("IVS_d_2D_calc"),
        cat_dset("IVS_cat").sort_index(level=1, key=demo_util.sorter),
        num_dset("RWT", fmt=demo_util._mean_sd_fmt_2),
        cat_dset("RWT_cat"),
        cat_dset("htn"),
    ]
    return pd.concat(demo_table_list)

In [ ]:
demo_table_2 = demo_table(df=dset_analytical_echo)
print(demo_table_2)

# Supplemental

## Table 1 Demographics EchoNet-LVH

In [ ]:
supp_demo_table1 = demo_table(df=dset_analytical_echo, missing_pred="enet")
print(supp_demo_table1)

## Table 2 Demographics Ultromics

In [ ]:
supp_demo_table2 = demo_table(df=dset_analytical_echo, missing_pred="ult_c")
print(supp_demo_table2)